## Gene Expression Analysis using Python

This notebook will show the implementation of artificial neural network for cancer gene expression classification.
Before passing the data throguh neural network we will use Principal Component Analysis to reduce dimensionality.

#### Dataset for this project comes from: https://www.kaggle.com/crawford/gene-expression

#### First we will import the nesscecary libraries for reading processing our dataset

In [1]:
import os #IO functions
import pandas as pd # data preprocessing
import numpy as np # linear algebra
import plotly.plotly as py # data visualization

### Content of our dataset (description from Kaggle)

Golub et al "Molecular Classification of Cancer: Class Discovery and Class Prediction by Gene Expression Monitoring"

There are two datasets containing the initial (training, 38 samples) and independent (test, 34 samples) datasets used in the paper. These datasets contain measurements corresponding to ALL and AML samples from Bone Marrow and Peripheral Blood. Intensity values have been re-scaled such that overall intensities for each chip are equivalent.

In [2]:
MAIN_DIR = os.getcwd()

X_train = pd.read_csv(os.path.join(MAIN_DIR, 'data_set_ALL_AML_train.csv'), encoding = 'utf-8')
y_train = pd.read_csv(os.path.join(MAIN_DIR,'actual.csv'))

print(f'Shape of X data: {X_train.shape}')
print(f'Shape of y data: {y_train.shape}')

X_train.head()

Shape of X data: (7129, 78)
Shape of y data: (72, 2)


,Gene Description,Gene Accession Number,1,call,2,call.1,3,call.2,4,call.3,...,29,call.33,30,call.34,31,call.35,32,call.36,33,call.37
0,AFFX-BioB-5_at (endogenous control),AFFX-BioB-5_at,-214,A,-139,A,-76,A,-135,A,...,15,A,-318,A,-32,A,-124,A,-135,A
1,AFFX-BioB-M_at (endogenous control),AFFX-BioB-M_at,-153,A,-73,A,-49,A,-114,A,...,-114,A,-192,A,-49,A,-79,A,-186,A
2,AFFX-BioB-3_at (endogenous control),AFFX-BioB-3_at,-58,A,-1,A,-307,A,265,A,...,2,A,-95,A,49,A,-37,A,-70,A
3,AFFX-BioC-5_at (endogenous control),AFFX-BioC-5_at,88,A,283,A,309,A,12,A,...,193,A,312,A,230,P,330,A,337,A
4,AFFX-BioC-3_at (endogenous control),AFFX-BioC-3_at,-295,A,-264,A,-376,A,-419,A,...,-51,A,-139,A,-367,A,-188,A,-407,A


### Data preprocessing 

First thing we have to do before implementing the PCA algorithm is to prepare our dataset.
PCA requires from us to pass it only a numeric matrix. So in order to create it we have to
delete all the unnesscecary columns from it.

#### 1. We will drop the first two columns of our dataset since they do not provide any information helping us with our classification task

In [3]:
X_train = X_train.iloc[:,2:]
X_train.head()

,1,call,2,call.1,3,call.2,4,call.3,5,call.4,...,29,call.33,30,call.34,31,call.35,32,call.36,33,call.37
0,-214,A,-139,A,-76,A,-135,A,-106,A,...,15,A,-318,A,-32,A,-124,A,-135,A
1,-153,A,-73,A,-49,A,-114,A,-125,A,...,-114,A,-192,A,-49,A,-79,A,-186,A
2,-58,A,-1,A,-307,A,265,A,-76,A,...,2,A,-95,A,49,A,-37,A,-70,A
3,88,A,283,A,309,A,12,A,168,A,...,193,A,312,A,230,P,330,A,337,A
4,-295,A,-264,A,-376,A,-419,A,-230,A,...,-51,A,-139,A,-367,A,-188,A,-407,A


#### 2. Next we will drop all the columns with 'call' header since its useless for us

In [4]:
X_train = X_train.drop(columns=[col for col in X_train.columns if 'call' in col])
X_train.head()

,1,2,3,4,5,6,7,8,9,10,...,35,36,37,38,28,29,30,31,32,33
0,-214,-139,-76,-135,-106,-138,-72,-413,5,-88,...,7,-213,-25,-72,-4,15,-318,-32,-124,-135
1,-153,-73,-49,-114,-125,-85,-144,-260,-127,-105,...,-100,-252,-20,-139,-116,-114,-192,-49,-79,-186
2,-58,-1,-307,265,-76,215,238,7,106,42,...,-57,136,124,-1,-125,2,-95,49,-37,-70
3,88,283,309,12,168,71,55,-2,268,219,...,132,318,325,392,241,193,312,230,330,337
4,-295,-264,-376,-419,-230,-272,-399,-541,-210,-178,...,-377,-209,-396,-324,-191,-51,-139,-367,-188,-407


In [5]:
X_train.describe()

,1,2,3,4,5,6,7,8,9,10,...,35,36,37,38,28,29,30,31,32,33
count,7129.000000,7129.000000,7129.000000,7129.000000,7129.000000,7129.000000,7129.000000,7129.000000,7129.000000,7129.000000,...,7129.000000,7129.000000,7129.000000,7129.000000,7129.000000,7129.000000,7129.000000,7129.000000,7129.000000,7129.000000
mean,641.367092,690.246318,698.307897,600.985271,679.532894,564.797728,584.437649,571.359097,789.713705,599.483097,...,514.496704,775.143498,689.248141,626.885959,673.279422,556.463179,718.934493,598.648899,676.920887,723.563473
std,2264.294361,2468.814372,2485.656277,2340.047428,2375.895416,2494.604090,2412.812263,2378.780450,2580.157021,2421.156219,...,2440.722824,2676.664777,2543.537830,2473.180838,2413.149603,2376.681824,2533.678058,2405.268550,2436.964933,2507.382019
min,-19826.000000,-17930.000000,-27182.000000,-23396.000000,-10339.000000,-21658.000000,-24024.000000,-27570.000000,-25171.000000,-12500.000000,...,-16281.000000,-27398.000000,-23673.000000,-23645.000000,-20376.000000,-9501.000000,-17580.000000,-25491.000000,-28400.000000,-27811.000000
25%,-21.000000,-14.000000,-31.000000,-33.000000,8.000000,-26.000000,-33.000000,-58.000000,-14.000000,-15.000000,...,-43.000000,-27.000000,-23.000000,-22.000000,-16.000000,-13.000000,-25.000000,-32.000000,-22.000000,-38.000000
50%,159.000000,130.000000,177.000000,139.000000,146.000000,106.000000,134.000000,140.000000,166.000000,103.000000,...,108.000000,144.000000,134.000000,133.000000,150.000000,82.000000,128.000000,107.000000,155.000000,170.000000
75%,535.000000,488.000000,610.000000,497.000000,471.000000,401.000000,497.000000,527.000000,609.000000,386.000000,...,396.000000,569.000000,505.000000,490.000000,517.000000,309.000000,488.000000,443.000000,549.000000,649.000000
max,31086.000000,29288.000000,28056.000000,31449.000000,29543.000000,38467.000000,41911.000000,40065.000000,23602.000000,28033.000000,...,61228.000000,37164.000000,32204.000000,29169.000000,29833.000000,30354.000000,25055.000000,28350.000000,25093.000000,32946.000000


### Fantastic, now we are ready for implementing our PCA

#### Principal Component Analysis 


    1. Standarize the d-dimensional data using formula

$$s= \frac {value - mean}{std}$$ 

    2. Eigendecomposition - get eigenvectors and eigenvalues
        - using correlation/covariance matrix, the covariance 
          between two features is defined as follows:
        
\begin{equation*}
\sigma_{jk} = \frac{1}{n-1}\sum_{i=1}^{N}(x_{ij} - \bar{x_j})(x_{ik} -\bar{x_k})
\end{equation*}

        This can be summarized via the following equation:
        
\begin{equation*}
\sigma_{jk} = \frac{1}{n-1}((X - \bar{x})^T(X - \bar{x}))
\end{equation*}
        
        Where mean vector is:
\begin{equation*} \bar{x} = \sum_{k=1}^n x_i \end{equation*}

        The mean vector is a d-dimensional vector where each value in this vector represents the sample mean of a feature column in the dataset.
        
        - using Singular Value Decomposition
        
    3. Sort eigenvalues in decreasing order then
       take the k eigenvectors coreesponding to
       the k highest eigenvalues.
       k - number of dimensions of our subspace
       
    4. Create the projection matrix W from k eigenvectors
    
    5. Transform the original d-dimensional data with projection
       matrix W in order to get subspace representation of our data
       

In [6]:
from sklearn.preprocessing import StandardScaler 


class Principal_Component_Analysis:
    
    def __init__(self, X):
        
        """
        Description:
            Constructor of PCA 

        Arguments:
            X - data to be processed

        Returns:
            Nothing, only sets the parameters for the given object.
        """

        self.X = X # data
        self.scaler = StandardScaler() # standarization
        self.explainde_variance = None

    def standarize(self):
        
        """
        Description:
            Method for data standarization, in this case
            we use Standard Scaler object from scikit-learn library

        Returns:
            Scaled X data.
        """

        return self.scaler.fit_transform(self.X)
    
    def create_covariance_matrix(self, X_scaled):
        
        """
        Description:
            This methods uses scaled data to create covariance matrix

        Arguments:
            X_scaled - standarized data

        Returns:
            Covariance matrix
        """

        mean_vec = np.mean(X_scaled, axis=0)
        covariance_matrix = (X_scaled - mean_vec).T.dot((X_scaled - mean_vec)) / (X_scaled.shape[0]-1)

        print(f'The covariance matrix: \n {covariance_matrix}')

        return covariance_matrix
    
    def eigendecomposion(self, covariance_matrix):

        """
        Description:
            This methods calculates the eigenvectors and eigenvalue from covariance matrix
            next it sorts the eigenvector and eigenvalues pairs in descending order.

        Arguments:
            covariance_matrix 

        Returns:
            List of eigenvectors and eigenvalues pairs in descending order

        """

        eig_vals, eig_vecs = np.linalg.eig(covariance_matrix)
        
        return eig_vals, eig_vecs
    


    def pair_eigen(self, eig_vals, eig_vecs):
        
        # Make a list of (eigenvalue, eigenvector) tuples
        eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

        # Sort the (eigenvalue, eigenvector) tuples from high to low
        eig_pairs.sort()
        eig_pairs.reverse()

        return eig_pairs
        
    def create_projection_matrix(self, eigenvectors):
        
        return matrix_w = np.hstack((eig_pairs[0][1].reshape(4,1), eig_pairs[1][1].reshape(4,1)))
    
    
    def project(self, projection_matrix):
        pass
    
    

In [7]:
pca = Principal_Component_Analysis(X=X_train)

In [8]:
scaled_data = pca.standarize()
scaled_data

D:\Anaconda\envs\tf_gpu\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning:

Data with input dtype int64 were all converted to float64 by StandardScaler.

D:\Anaconda\envs\tf_gpu\lib\site-packages\sklearn\base.py:464: DataConversionWarning:

Data with input dtype int64 were all converted to float64 by StandardScaler.



array([[-0.3777897 , -0.33591204, -0.3115323 , ..., -0.26221319,
        -0.32867813, -0.34243832],
       [-0.35084785, -0.30917669, -0.30066921, ..., -0.2692815 ,
        -0.31021124, -0.36277969],
       [-0.30888923, -0.28001085, -0.40447202, ..., -0.22853475,
        -0.29297548, -0.31651305],
       ...,
       [-0.26737228, -0.27514987, -0.26445893, ..., -0.25971849,
        -0.26178696, -0.29337973],
       [-0.19891348, -0.2488196 , -0.18922201, ..., -0.19360897,
        -0.15549976,  0.02649823],
       [-0.29961417, -0.2852769 , -0.29745052, ..., -0.26221319,
        -0.27902272, -0.29258203]])

In [9]:
cov_matrix = pca.create_covariance_matrix(X_scaled=scaled_data)

The covariance matrix: 
 [[1.00014029 0.91503972 0.90718432 ... 0.92064222 0.92828757 0.89094937]
 [0.91503972 1.00014029 0.9187555  ... 0.91147706 0.93310085 0.90020722]
 [0.90718432 0.9187555  1.00014029 ... 0.90103848 0.89669849 0.85439033]
 ...
 [0.92064222 0.91147706 0.90103848 ... 1.00014029 0.94553219 0.90959979]
 [0.92828757 0.93310085 0.89669849 ... 0.94553219 1.00014029 0.92959839]
 [0.89094937 0.90020722 0.85439033 ... 0.90959979 0.92959839 1.00014029]]


In [10]:
eig_vals, eig_vecs = pca.eigendecomposion(cov_matrix)

In [11]:
tot = sum(eig_vals)
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

trace1 = dict(type='bar',x=['PC %s' %i for i in range(1,5)],y=var_exp,name='Individual')
trace2 = dict(type='scatter',x=['PC %s' %i for i in range(1,5)], y=cum_var_exp,name='Cumulative')

data = [trace1, trace2]

layout=dict(title='Explained variance by different principal components',yaxis=dict(title='Explained variance in percent'),
annotations=list([
    dict(
        x=1.16,
        y=1.05,
        xref='paper',
        yref='paper',
        text='Explained Variance',
        showarrow=False)]))

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='selecting-principal-components')

D:\Anaconda\envs\tf_gpu\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [12]:
eig_pairs = pca.pair_eigen(eig_vals, eig_vecs)

In [18]:
print(eig_pairs[0][1].shape)

(38,)
